# Exploratory Data Analysis

In [9]:
import pandas as pd

input_data_path = '../input_data/'
features_df = pd.read_csv(f'{input_data_path}features.csv')
stores_df = pd.read_csv(f'{input_data_path}stores.csv')
weekly_sales_df = pd.read_csv(f'{input_data_path}weekly_sales.csv')

Primero limpiar data según requerimientos: **tiendas tipo B, con tamaño mayor a 100.000 y semanas con temperaturas mayores a 25°**

In [33]:
# Tipo B y tamaño mayor a 100k
valid_stores = stores_df.query("Type == 'B' and Size > 100000").Store.values

Rango de fechas y filtro de semanas por temperatura

In [43]:
print(pd.to_datetime(weekly_sales_df['Date']).max())
print(pd.to_datetime(weekly_sales_df['Date']).min())

2012-10-26 00:00:00
2010-02-05 00:00:00


In [46]:
print(pd.to_datetime(features_df['Date']).max())
print(pd.to_datetime(features_df['Date']).min())

2013-07-26 00:00:00
2010-02-05 00:00:00


Para el dataset de ventas, tenemos registros desde 2010-02-05 hasta 2012-10-26 (viernes).
En el dataset de Features, tenemos registros desde 2010-02-05 hasta 2013-07-26 (viernes).
Pero debemos pronosticar las siguientes 7 semanas, es decir, hasta el 2012-12-14 (viernes).
Entonces vamos a limitar nuestro dataset de features entre las fechas 2010-02-05 y 2012-12-14.
Además vamos a eliminar los registros en donde la temperatura sea igual o menor a 25

In [47]:
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [55]:
features_clean_df = features_df.query("Store in @valid_stores").copy(deep=True)
features_clean_df['Date'] = pd.to_datetime(features_clean_df['Date']) # Cast Date into datetime
features_clean_df = features_clean_df.query("'2010-02-05' <= Date <= '2012-12-14'") # Filtro de fechas
features_clean_df = features_clean_df.query("Temperature > 25")

In [67]:
survived = round(100 * (1 - (features_clean_df.shape[0] / features_df.shape[0])), 1)
print(f"El número de registros en el dataset de features disminuyó un {survived}% con los filtros aplicados")

El número de registros en el dataset de features disminuyó un 80.5% con los filtros aplicados


In [69]:
features_clean_df

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
1456,9,2010-02-05,38.01,2.572,NaN,NaN,NaN,NaN,NaN,214.655459,6.415,False
1457,9,2010-02-12,37.08,2.548,NaN,NaN,NaN,NaN,NaN,214.805653,6.415,True
1458,9,2010-02-19,43.06,2.514,NaN,NaN,NaN,NaN,NaN,214.850618,6.415,False
1459,9,2010-02-26,43.83,2.561,NaN,NaN,NaN,NaN,NaN,214.878045,6.415,False
1460,9,2010-03-05,48.43,2.625,NaN,NaN,NaN,NaN,NaN,214.905472,6.415,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8153,45,2012-11-16,45.61,3.750,18700.38,499.75,28.20,1800.78,10303.93,192.261619,8.667,False
8154,45,2012-11-23,43.08,3.748,1005.79,NaN,72542.01,484.70,620.12,192.283032,8.667,True
8155,45,2012-11-30,37.43,3.729,2298.55,NaN,2908.54,100.86,1344.84,192.304445,8.667,False
8156,45,2012-12-07,45.53,3.688,9273.32,NaN,263.41,1979.78,11324.83,192.325858,8.667,False


Se observa que existen muchos valores NaN en las columnas MarkDown, cuantos NaN existen por columna?

In [70]:
features_clean_df.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1        967
MarkDown2       1136
MarkDown3       1011
MarkDown4        969
MarkDown5        967
CPI                0
Unemployment       0
IsHoliday          0
dtype: int64

Existe un alto número de valores nulos en MarkDown1-5, es posible que en esas semanas no existieran promociones y descuentos, podriamos reemplazar esos valores por 0. Utilizaremos este supuesto más adelante

## Limpieza de Ventas Semanales (tiendas y semanas)

In [79]:
clean_weekly_sales = weekly_sales_df.query("Store in @valid_stores").copy(deep=True)
clean_weekly_sales['Date'] = pd.to_datetime(clean_weekly_sales['Date']) # Cast Date into datetime

In [91]:
clean_weekly_sales

,Store,Dept,Date,Weekly_Sales,IsHoliday
78657,9,1,2010-02-05,12861.40,False
78658,9,1,2010-02-12,20273.94,True
78659,9,1,2010-02-19,14819.97,False
78660,9,1,2010-02-26,10530.98,False
78661,9,1,2010-03-05,10438.47,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


No tenemos features específicas para Dept (departamentos), es posible que sea mejor realizar la predicciónes por las tiendas completas, y no una para cada departamento.

In [97]:
store_weekly_sales = clean_weekly_sales.groupby(['Store', 'Date', 'IsHoliday']).sum().reset_index().drop(columns=['Dept'])

In [102]:
# Revisar si existen pares duplicados [Store, Date] (por algún error en la columna IsHoliday)
store_weekly_sales[store_weekly_sales.duplicated(subset=['Store','Date'], keep=False)]

,Store,Date,IsHoliday,Weekly_Sales


## Algunos gráficos sobre ventas por departamento y tienda

In [103]:
clean_weekly_sales

,Store,Dept,Date,Weekly_Sales,IsHoliday
78657,9,1,2010-02-05,12861.40,False
78658,9,1,2010-02-12,20273.94,True
78659,9,1,2010-02-19,14819.97,False
78660,9,1,2010-02-26,10530.98,False
78661,9,1,2010-03-05,10438.47,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False
